### 영어로 질문을 받으면 한글로 답을 주는
### 수정 :  한국어를 영어로 번역

1. 기반 종류
    1. 번역기 기반
    - google translator : 무료, 빠름, 성능 안좋음
    - YandexTranslator : 무료, 인터넷에서 사용해봤으나 성능 안좋음
    - DeepLTranslator : 무료, 가입불가
    - LibreTranslate : 무료, 인터넷에서 사용해봤으나 성능 안좋음
    - Microsoft translator TEXT API : 하루 200만자 무료

    2. 모델 기반
    - MBartForConditionalGeneration : 실행시간(한 문장당 약 6.3초)
    - MarianMTModel : 실행시간(한 문장당 약 4.73초)
    - T5ForConditionalGeneration : 한글깨짐
    - Bart : 한글깨짐
    - NLLB : 한글깨짐
 
2. Chat gpt 의견 
- 문장 해석 : Microsoft translator TEXT API 결과가 나음
- 단어 해석 : MarianMTModel 결과가 나음

3. 최종 시도 모델
- google translator
- MBartForConditionalGeneration
- MarianMTModel
- NLLB
- Microsoft translator TEXT API

In [ ]:
# google translator
import time
from googletrans import Translator
from requests.exceptions import Timeout

translator = Translator()


def translation_kor_to_eng(content_kor):
    result = translator.translate(content_kor, src="ko", dest="en")
    return result.text


def translation_eng_to_kor(content_eng):
    result = translator.translate(content_eng, src="en", dest="ko")
    return result.text


# 실행결과 0.9초
# test1 = translation_kor_to_eng("각을 직접 그리는 방법을 배우려면 어떤 강의를 들어야해?")
# test2 = translation_kor_to_eng("예각과 둔각을 이해하려면 어떤 강의를 들어야해?")
# test3 = translation_kor_to_eng(
#     "각도기를 이용한 각의 크기 재는 방법 학습하려면 어떤 강의를 들어야해?"
# )

# 실행결과 0.9초
test1 = translation_kor_to_eng(
    "사각형, 내각, 합, 360도, 각도기, 삼각형, 문제풀이, 더하기, 빼기"
)
test2 = translation_kor_to_eng(
    "각, 각도기, 자, 예각, 둔각, 직각, 그리기, 눈금, 문제, 연습"
)
test3 = translation_kor_to_eng("나누기, 몫, 나머지, 곱셈, 계산")


print(test1)
print(test2)
print(test3)

Rectangular, cabinet, sum, 360 degrees, angle, triangle, problem solving, plus, subtraction
Angle, angle, chamber, angle, dull angle, right angle, drawing, grid, problem, practice
Sharing, share, rest, multiplication, calculation


In [14]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


def load_model():
    model = MBartForConditionalGeneration.from_pretrained(
        "facebook/mbart-large-50-many-to-many-mmt"
    )
    tokenizer = MBart50TokenizerFast.from_pretrained(
        "facebook/mbart-large-50-many-to-many-mmt"
    )
    return model, tokenizer


def translate_korean_to_english(text):
    model, tokenizer = load_model()
    tokenizer.src_lang = "ko_KR"
    encoded_text = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_text, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


# 실행결과 19.2초
# test4 = translate_korean_to_english(
#     "각을 직접 그리는 방법을 배우려면 어떤 강의를 들어야해?"
# )
# test5 = translate_korean_to_english("예각과 둔각을 이해하려면 어떤 강의를 들어야해?")
# test6 = translate_korean_to_english("각도기를 이용한 각의 크기 재는 방법 학습하려면 어떤 강의를 들어야해?")

# 실행결과 19.7초
test4 = translate_korean_to_english(
    "사각형, 내각, 합, 360도, 각도기, 삼각형, 문제풀이, 더하기, 빼기"
)
test5 = translate_korean_to_english(
    "각, 각도기, 자, 예각, 둔각, 직각, 그리기, 눈금, 문제, 연습"
)
test6 = translate_korean_to_english("나누기, 몫, 나머지, 곱셈, 계산")

print(test4)
print(test5)
print(test6)

c:\Users\user\anaconda3\envs\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Square, rectangle, join, 360 degrees, angle, triangle, problem solving, addition, subtraction.
Angle, angle, yes, angle, angle, square, rectangle, drawing, glue, problem, practice.
Divide, divide, divide, multiply, calculate.


In [23]:
import os
from transformers import MarianMTModel, MarianTokenizer

# 텍스트를 파일로 저장
def save_transcript_to_file(transcript, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(transcript)

# 폴더 생성
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# MarianMT 한국어-영어 번역 모델 불러오기
def load_marian_model():
    model_name = "Helsinki-NLP/opus-mt-ko-en"  # MarianMT 한국어-영어 모델
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return model, tokenizer

# MarianMT를 사용하여 한국어 텍스트를 영어로 번역
def translate_korean_to_english_marian(text):
    model, tokenizer = load_marian_model()
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# 텍스트 파일에서 "키워드:" 부분만 추출
def extract_keywords_from_text(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            if "키워드:" in content:
                # "키워드:" 이후의 텍스트를 추출
                keywords = content.split("키워드:")[1].strip()
                return keywords
            else:
                print(f"'키워드:'를 찾을 수 없습니다: {file_path}")
                return None
    except Exception as e:
        print(f"텍스트 파일을 읽는 중 오류 발생: {e}")
        return None

# 폴더 내 모든 텍스트 파일에서 "키워드:" 부분만 번역
def text_translator(folder_path):
    text_folder_path = os.path.join(folder_path, "translator2")
    create_folder_if_not_exists(text_folder_path)
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # .txt 파일만 처리
            text_path = os.path.join(folder_path, filename)
            text_file_path = os.path.join(
                text_folder_path, f"{os.path.splitext(filename)[0]}_translated.txt"
            )
            # "키워드:" 부분만 추출
            keywords = extract_keywords_from_text(text_path)
            if keywords:
                # 추출한 키워드 번역
                translated_keywords = translate_korean_to_english_marian(keywords)
                # 번역된 키워드를 텍스트 파일로 저장
                save_transcript_to_file(translated_keywords, text_file_path)
                print(f"Translated keywords for {filename} saved to {text_file_path}")

# 예시 사용
folder_path = "./lecture_video/audio/summary/4_1"
text_translator(folder_path)


Translated keywords for 01강큰수(1)_만점왕수학4-1(2024)_transcript_processed.txt saved to ./lecture_video/audio/summary/4_1\translator2\01강큰수(1)_만점왕수학4-1(2024)_transcript_processed_translated.txt
Translated keywords for 02강큰수(2)_만점왕수학4-1(2024)_transcript_processed.txt saved to ./lecture_video/audio/summary/4_1\translator2\02강큰수(2)_만점왕수학4-1(2024)_transcript_processed_translated.txt
Translated keywords for 03강큰수(3)_만점왕수학4-1(2024)_transcript_processed.txt saved to ./lecture_video/audio/summary/4_1\translator2\03강큰수(3)_만점왕수학4-1(2024)_transcript_processed_translated.txt
Translated keywords for 04강큰수(4)_만점왕수학4-1(2024)_transcript_processed.txt saved to ./lecture_video/audio/summary/4_1\translator2\04강큰수(4)_만점왕수학4-1(2024)_transcript_processed_translated.txt
Translated keywords for 06강각도(1)_만점왕수학4-1(2024)_transcript_processed.txt saved to ./lecture_video/audio/summary/4_1\translator2\06강각도(1)_만점왕수학4-1(2024)_transcript_processed_translated.txt
Translated keywords for 07강각도(2)_만점왕수학4-1(2024)_transcript_p

In [24]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

# MarianMT 한국어-영어 번역 모델 불러오기
def load_marian_model():
    model_name = "Helsinki-NLP/opus-mt-ko-en"  # MarianMT 한국어-영어 모델
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return model, tokenizer

# MarianMT를 사용하여 한국어 텍스트를 영어로 번역
def translate_korean_to_english_marian(text):
    model, tokenizer = load_marian_model()
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# CSV 파일에서 'lecture_keywords' 열을 번역하여 'eng_keywords' 열에 저장
def translate_keywords_in_csv(csv_file_path):
    try:
        # CSV 파일 읽기
        df = pd.read_csv(csv_file_path)
        
        # 'lecture_keywords' 열이 있는지 확인
        if 'lecture_keywords' not in df.columns:
            print(f"'lecture_keywords' 열이 CSV 파일에 존재하지 않습니다.")
            return
        
        # 'lecture_keywords' 열을 번역하여 'eng_keywords' 열에 저장
        translated_keywords = []
        for keywords in df['lecture_keywords']:
            if pd.isnull(keywords):  # 결측치 처리
                translated_keywords.append(None)
            else:
                # 키워드 번역
                translated_text = translate_korean_to_english_marian(keywords)
                translated_keywords.append(translated_text)
        
        # 새로운 'eng_keywords' 열 추가
        df['eng_keywords'] = translated_keywords
        
        # 기존 CSV 파일을 덮어쓰면서 저장
        df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')
        print(f"CSV 파일에 'eng_keywords' 열이 추가되었습니다: {csv_file_path}")
    
    except Exception as e:
        print(f"CSV 파일을 처리하는 중 오류 발생: {e}")

# 예시 사용
csv_file_path = './lecture_video/audio/lecture.csv'  # 입력 CSV 파일 경로
translate_keywords_in_csv(csv_file_path)




c:\Users\user\anaconda3\envs\myenv\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
c:\Users\user\anaconda3\envs\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


CSV 파일에 'eng_keywords' 열이 추가되었습니다: ./lecture_video/audio/lecture.csv


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


def load_nllb_model():
    model_name = "facebook/nllb-200-distilled-600M"  # NLLB 모델
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return model, tokenizer


def translate_korean_to_english_nllb(text):
    model, tokenizer = load_nllb_model()

    # 입력 텍스트를 처리하는 부분 (언어 코드를 명시하지 않음)
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    # 번역 작업 실행 (forced_bos_token_id 사용하지 않음)
    translated = model.generate(**inputs, max_length=100)

    return tokenizer.decode(translated[0], skip_special_tokens=True)


# 테스트
test10 = translate_korean_to_english_nllb(
    "각을 직접 그리는 방법을 배우려면 어떤 강의를 들어야해?"
)
test11 = translate_korean_to_english_nllb(
    "예각과 둔각을 이해하려면 어떤 강의를 들어야해?"
)
test12 = translate_korean_to_english_nllb(
    "각도기를 이용한 각의 크기 재는 방법 학습하려면 어떤 강의를 들어야해?"
)

print(test10)
print(test11)
print(test12)

¿Qué clase tienes que tener para aprender a dibujar directamente?
¿Qué clase de clases hay que escuchar para entender el sentido y la esencia?
Welche Vorlesungen müssen Sie hören, um zu lernen, wie man die Größe eines einzelnen Zimmers neu bestimmen kann?


In [ ]:
# 182afd49afb44b359934b0bbf4e85182
# https://translator-api-mini-proj.cognitiveservices.azure.com
import requests
import uuid
import json

# Azure Translator API 설정
subscription_key = "YOUR_API_KEY"  # API 키 입력
endpoint = "https://api.cognitive.microsofttranslator.com"  # 엔드포인트 확인
location = "koreacentral"  # 리소스 위치 설정

In [17]:
# 번역 함수 (한국어 -> 영어)
def translation_kor_to_eng(content_kor):
    path = "/translate?api-version=3.0"
    constructed_url = endpoint + path

    headers = {
        "Ocp-Apim-Subscription-Key": subscription_key,
        "Ocp-Apim-Subscription-Region": location,
        "Content-type": "application/json",
        "X-ClientTraceId": str(uuid.uuid4()),
    }

    # 요청 바디에 번역할 내용을 넣음
    body = [{"text": content_kor}]

    params = {"from": "ko", "to": "en"}

    response = requests.post(constructed_url, headers=headers, params=params, json=body)

    try:
        response.raise_for_status()  # 상태 코드 확인 (200 OK)
        result = response.json()
        print("Response JSON:", result)  # 응답 내용 확인
        return result[0]["translations"][0]["text"]
    except KeyError as e:
        return f"KeyError: {e}, response content: {response.json()}"
    except Exception as e:
        return f"An error occurred: {e}"

In [18]:
# 실행결과 0초
# test13 = translation_kor_to_eng("각을 직접 그리는 방법을 배우려면 어떤 강의를 들어야해?")
# test14 = translation_kor_to_eng("예각과 둔각을 이해하려면 어떤 강의를 들어야해?")
# test15 = translation_kor_to_eng("각도기를 이용한 각의 크기 재는 방법 학습하려면 어떤 강의를 들어야해?")

test13 = translation_kor_to_eng(
    "사각형, 내각, 합, 360도, 각도기, 삼각형, 문제풀이, 더하기, 빼기"
)
test14 = translation_kor_to_eng(
    "각, 각도기, 자, 예각, 둔각, 직각, 그리기, 눈금, 문제, 연습"
)
test15 = translation_kor_to_eng("나누기, 몫, 나머지, 곱셈, 계산")

Response JSON: [{'translations': [{'text': 'Rectangle, Cabinet, Sum, 360 degrees, Protractor, Triangle, Problem Solving, Addition, Subtraction', 'to': 'en'}]}]
Response JSON: [{'translations': [{'text': 'angle, protractor, ruler, acute angle, obtuse angle, right angle, drawing, grid, problem, practice', 'to': 'en'}]}]
Response JSON: [{'translations': [{'text': 'Division, quotient, remainder, multiplication, calculation', 'to': 'en'}]}]


In [19]:
# 실행결과 0초
print(test13)
print(test14)
print(test15)

Rectangle, Cabinet, Sum, 360 degrees, Protractor, Triangle, Problem Solving, Addition, Subtraction
angle, protractor, ruler, acute angle, obtuse angle, right angle, drawing, grid, problem, practice
Division, quotient, remainder, multiplication, calculation


In [22]:
import os
from transformers import MarianMTModel, MarianTokenizer

# 텍스트를 파일로 저장
def save_transcript_to_file(transcript, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(transcript)

# 폴더 생성
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)


# 텍스트 파일에서 "키워드:" 부분만 추출
def extract_keywords_from_text(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            if "키워드:" in content:
                # "키워드:" 이후의 텍스트를 추출
                keywords = content.split("키워드:")[1].strip()
                return keywords
            else:
                print(f"'키워드:'를 찾을 수 없습니다: {file_path}")
                return None
    except Exception as e:
        print(f"텍스트 파일을 읽는 중 오류 발생: {e}")
        return None

# 폴더 내 모든 텍스트 파일에서 "키워드:" 부분만 번역
def text_translator(folder_path):
    text_folder_path = os.path.join(folder_path, "translator")
    create_folder_if_not_exists(text_folder_path)
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # .txt 파일만 처리
            text_path = os.path.join(folder_path, filename)
            text_file_path = os.path.join(
                text_folder_path, f"{os.path.splitext(filename)[0]}_translated.txt"
            )
            # "키워드:" 부분만 추출
            keywords = extract_keywords_from_text(text_path)
            if keywords:
                # 추출한 키워드 번역
                translated_keywords = translation_kor_to_eng(keywords)
                # 번역된 키워드를 텍스트 파일로 저장
                save_transcript_to_file(translated_keywords, text_file_path)
                print(f"Translated keywords for {filename} saved to {text_file_path}")

# 예시 사용
folder_path = "./lecture_video/audio/summary/4_1"
text_translator(folder_path)


Response JSON: [{'translations': [{'text': 'Million, five-digit, digit value, number reading', 'to': 'en'}]}]
Translated keywords for 01강큰수(1)_만점왕수학4-1(2024)_transcript_processed.txt saved to ./lecture_video/audio/summary/4_1\translator\01강큰수(1)_만점왕수학4-1(2024)_transcript_processed_translated.txt
Response JSON: [{'translations': [{'text': '100,000, 1,000,000, 10,000,000, digits, large', 'to': 'en'}]}]
Translated keywords for 02강큰수(2)_만점왕수학4-1(2024)_transcript_processed.txt saved to ./lecture_video/audio/summary/4_1\translator\02강큰수(2)_만점왕수학4-1(2024)_transcript_processed_translated.txt
Response JSON: [{'translations': [{'text': 'Billions, trillions, large numbers, digits', 'to': 'en'}]}]
Translated keywords for 03강큰수(3)_만점왕수학4-1(2024)_transcript_processed.txt saved to ./lecture_video/audio/summary/4_1\translator\03강큰수(3)_만점왕수학4-1(2024)_transcript_processed_translated.txt
Response JSON: [{'translations': [{'text': 'jumping, million, billion, trillion, number size comparison, digits, large